# GEE Aster Preprocessing

##Step 1: Installation


In [1]:
!pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git

  Running command git clone --filter=blob:none --quiet https://github.com/Mining-for-the-Future/ASTER_preprocessing.git 'C:\Users\eweav\AppData\Local\Temp\pip-req-build-bjgpcec5'

[notice] A new release of pip is available: 23.1.1 -> 23.3.2
[notice] To update, run: C:\Users\eweav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Cloning https://github.com/Mining-for-the-Future/ASTER_preprocessing.git to c:\users\eweav\appdata\local\temp\pip-req-build-bjgpcec5
  Resolved https://github.com/Mining-for-the-Future/ASTER_preprocessing.git to commit 43e33075dafd7434646e9c664b2a4293c86f8c53
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached ee-0.2-py3-none-any.whl
  Using cached blessings-1.7-py3-none-any.whl (18 kB)
  Created wheel for ASTER_Preprocessing: filename=ASTER_Preprocessing-0.1.0-py3-none-any.whl size=6069 sha256=6048efe9c453a3a440b455eaf4f87aa3f5cfd4705b96bb2dd065dcaff565708b
  Stored in directory: C:\Users\eweav\AppData\Local\Temp\pip-ephem-wheel-cache-_h_ug_pl\wheels\09\71\c4\6c644f802195eed30498458715efaa1e47f

##Step 2: Importing Libraries

In [5]:
import ASTER_preprocessing

In [2]:
import ee

ModuleNotFoundError: No module named '_curses'

In [ ]:
import geemap

In [4]:
from ASTER_preprocessing import preprocessing, data_conversion, masks

ModuleNotFoundError: No module named 'StringIO'

##Step 3: Google Earth Engine Authentication

In [7]:
ee.Authenticate()
ee.Initialize()

NameError: name 'ee' is not defined

##Step 4: Preprocess ASTER Imagery Using a Drawn Polygon

In [8]:
# Create an interactive map
Map = geemap.Map()

# Add drawing tools to the map
Map.addDrawingTools()

# Display the map
Map

NameError: name 'geemap' is not defined

In [ ]:
# Get the last drawn feature
geometry = Map.draw_last_feature

# Check if a geometry was drawn and it's a Polygon
if geometry is not None and geometry.geometry().type().getInfo() == 'Polygon':
    # Run the preprocessing
    result = preprocessing.aster_preprocessing(geometry.geometry())
    # `result` is a dictionary containing the preprocessed image and metadata
else:
    print("Please draw a polygon on the map.")